# ***Bike Sharing in New York, U.S 2019***

This is part of my journey towards mastering Python for Data Science. Below I documented few lessons learned through out the data cleaning/wrangling process.

I use United States bike sharing data (particularly New York City) for data cleaning; please visit [bikeshare.com](https://www.citibikenyc.com/system-data) for data.

## Python Library

In [1]:
# install haversine if required
# use memory_profiler to check memory usage for each command

!pip install haversine memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp37-none-any.whl size=30180 sha256=d483b655d6e5052b5a3c3c814acd087a7f4a3a50b7f7d9d043d9ea296f6cf4de
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [1]:
# data analysis
import numpy as np
import pandas as pd

# Operating System
from glob import glob
from os import path
from pathlib import Path

# calculate distance between geo coordinates
from haversine import haversine_vector, Unit

%load_ext memory_profiler

## Lesson 01: Import CSV, systematically & efficiently

After trial & error, I find out various ways of import CSV, specifically multiple CSV files. Most courses teach ***Pandas read_csv()*** method for single file, and usually the file size is small for education purpose..

However, in reality, an analyst might has to gather multipe CSV files which each file could be huge in size. In this case, bike sharing monthly file is 100 MB+. We will need to analyze 12-month worth of data.


Here I demonstrate how the concept of _**Don't Repeat Yourself (DRY)**_ - avoid long & repetitive codes for similar task, is applicable in practical context.

In [ ]:
# list files available in the 'New York Bike Share' directory
# 13 files in the directory which we only need file in format "*-citibike-tripdata.csv" (i.e. 201901-citibike-tripdata.csv etc)
# each file is 100 MB+

!ls -lh /content/drive/MyDrive/'New York Bike Share'

total 5.4G
-rw------- 1 root root 176M Feb 11  2019 201901-citibike-tripdata.csv
-rw------- 1 root root 172M Mar  4  2019 201902-citibike-tripdata.csv
-rw------- 1 root root 242M Apr 15  2019 201903-citibike-tripdata.csv
-rw------- 1 root root 322M May  6  2019 201904-citibike-tripdata.csv
-rw------- 1 root root 351M Jun 11  2019 201905-citibike-tripdata.csv
-rw------- 1 root root 387M Jul 15  2019 201906-citibike-tripdata.csv
-rw------- 1 root root 397M Aug  5  2019 201907-citibike-tripdata.csv
-rw------- 1 root root 426M Sep 16  2019 201908-citibike-tripdata.csv
-rw------- 1 root root 443M Oct 11  2019 201909-citibike-tripdata.csv
-rw------- 1 root root 379M Nov  5  2019 201910-citibike-tripdata.csv
-rw------- 1 root root 268M Dec 20  2019 201911-citibike-tripdata.csv
-rw------- 1 root root 173M Jan 21  2020 201912-citibike-tripdata.csv
-rw------- 1 root root 1.7G Feb 19 02:04 new_york_bikeshare_2019.csv


### BEFORE: The Inefficient Way

In [ ]:
'''
DON'T RUN... For demo only...

Issue: repetitive codes that require copy + paste and are error prone.
Note: Imagine we have files for 5 years (60 months) within the same directory ??

file_01 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201901-citibike-tripdata.csv')
file_02 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201902-citibike-tripdata.csv')
file_03 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201903-citibike-tripdata.csv')
file_04 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201904-citibike-tripdata.csv')
file_05 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201905-citibike-tripdata.csv')
file_06 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201906-citibike-tripdata.csv')
file_07 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201907-citibike-tripdata.csv')
file_08 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201908-citibike-tripdata.csv')
file_09 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201909-citibike-tripdata.csv')
file_10 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201910-citibike-tripdata.csv')
file_11 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201911-citibike-tripdata.csv')
file_12 = pd.read_csv('/content/drive/MyDrive/New York Bike Share/201912-citibike-tripdata.csv')

df = pd.concat([file_01, file_02, file_03, ....], ignore_index=True)

'''

### AFTER: The Systematic & Efficient Way

In [2]:
'''
Step 1: Getting file names within directory that are required for analysis

'''

# replace 'file_dir' with your working directory
# current path of working directory for jupyter notebook and CSV files in Google Colab
file_dir = '/content/drive/MyDrive/New York Bike Share'

# getting file names within the directory
file_names = glob(path.join(file_dir, '*-citibike-tripdata.csv'))
file_names

['/content/drive/MyDrive/New York Bike Share/201901-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201902-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201903-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201904-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201905-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201906-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201907-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201908-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201909-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201910-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201911-citibike-tripdata.csv',
 '/content/drive/MyDrive/New York Bike Share/201912-citibike-tripdata.csv']

In [3]:
# only import 100 line items for quick view of column names and data type
# let's leave columns such as 'stop_time', station_id, station_name & bike_id out of this demo

pd.read_csv('/content/drive/MyDrive/New York Bike Share/201901-citibike-tripdata.csv', nrows=100).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tripduration             100 non-null    int64  
 1   starttime                100 non-null    object 
 2   stoptime                 100 non-null    object 
 3   start station id         100 non-null    int64  
 4   start station name       100 non-null    object 
 5   start station latitude   100 non-null    float64
 6   start station longitude  100 non-null    float64
 7   end station id           100 non-null    int64  
 8   end station name         100 non-null    object 
 9   end station latitude     100 non-null    float64
 10  end station longitude    100 non-null    float64
 11  bikeid                   100 non-null    int64  
 12  usertype                 100 non-null    object 
 13  birth year               100 non-null    int64  
 14  gender                   10

In [4]:
'''
Step 2: Define function to create Pandas DataFrame

'''

col_index = [0, 1, 5, 6, 9 ,10, 12, 13, 14]

col_name = ['duration', 
            'time_start', 
            'station_latitude_start', 
            'station_longitude_start',
            'station_latitude_end', 
            'station_longitude_end', 
            'user_type', 
            'birth_year', 
            'gender']

col_type = {
    'duration': np.int32,
    'station_latitude_start': np.float32,
    'station_longitude_start': np.float32,
    'station_latitude_end': np.float32,
    'station_longitude_end': np.float32,
    'user_type': 'category',
    'birth_year': np.int32,
    'gender': 'category'
}

# self defined function to create dataframe
def create_df(f, size = 100_000):

    # create chunks of data frame with 100K per chunk. Result is an iteratable of dataframes
    result = pd.read_csv(f, chunksize=size, usecols=col_index, names=col_name, dtype=col_type, parse_dates=['time_start'], header=0)

    return result

In [5]:
'''

Step 3: Use .map() to apply 'create_df' function to each file in 'file_names'

'''

# .map() will apply 'create_df' function to each file in file_names
# ref: https://realpython.com/python-map-function/#getting-started-with-pythons-map
# result >> list of iteratable. Each iterable contains many dataframes with 100,000 rows

df = list(map(create_df, file_names))
df

In [6]:
'''
Step 4: apply python 'list comprehension' to get list of DataFrame

'''

# loop through each iteratable and store each dataframe to list with 'list comprehension'

df = [chunk for ls in df for chunk in ls]

In [ ]:
# let's check one of the dataframe
# each dataframe contains up to 100,000 line items
# df[0] index number range from 0 to 99,999

df[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   duration                 100000 non-null  int32         
 1   time_start               100000 non-null  datetime64[ns]
 2   station_latitude_start   100000 non-null  float32       
 3   station_longitude_start  100000 non-null  float32       
 4   station_latitude_end     100000 non-null  float32       
 5   station_longitude_end    100000 non-null  float32       
 6   user_type                100000 non-null  category      
 7   birth_year               100000 non-null  int32         
 8   gender                   100000 non-null  category      
dtypes: category(2), datetime64[ns](1), float32(4), int32(2)
memory usage: 3.2 MB


In [ ]:
# df[1] index number range from 100,000 to 199,999

df[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 100000 to 199999
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   duration                 100000 non-null  int32         
 1   time_start               100000 non-null  datetime64[ns]
 2   station_latitude_start   100000 non-null  float32       
 3   station_longitude_start  100000 non-null  float32       
 4   station_latitude_end     100000 non-null  float32       
 5   station_longitude_end    100000 non-null  float32       
 6   user_type                100000 non-null  category      
 7   birth_year               100000 non-null  int32         
 8   gender                   100000 non-null  category      
dtypes: category(2), datetime64[ns](1), float32(4), int32(2)
memory usage: 3.2 MB


In [7]:
'''

Step 5: use pd.concat() to merge list of dataframes

'''

# p.concat() >> append list of dataframe on top of each other to produce master dataframe
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
# note: each dataframe has different index number. parameter 'ignore_index' is set to 'True' and pd.concat() will reset index number after merge.

df = pd.concat(df, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20551697 entries, 0 to 20551696
Data columns (total 9 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   duration                 int32         
 1   time_start               datetime64[ns]
 2   station_latitude_start   float32       
 3   station_longitude_start  float32       
 4   station_latitude_end     float32       
 5   station_longitude_end    float32       
 6   user_type                category      
 7   birth_year               int32         
 8   gender                   category      
dtypes: category(2), datetime64[ns](1), float32(4), int32(2)
memory usage: 666.4 MB


In [ ]:
# check variables with NaN items

df.isna().sum()

duration                   0
time_start                 0
station_latitude_start     0
station_longitude_start    0
station_latitude_end       0
station_longitude_end      0
user_type                  0
birth_year                 0
gender                     0
dtype: int64

## Lesson 02: Pandas 'Category' Data Type

Gender data type is set to 'category' when we import CSV with .read_csv() method. I learn that a category will remain in pandas memory/dataframe even I explicitly exclude the line items with gender category '0'.

We can check category of variable with ***.cat.categories*** attribute and remove unwanted category with ***.cat.remove_categories()*** method. Finally, we also can use ***.cat.rename_categories()*** method to rename category.

In [8]:
# Based on data dictionary, 'gender' consists category 0, 1, 2 which category 0 is 'unknown'
# .unique() method can easily shows unique category for 'gender'

df.gender.unique()

['1', '2', '0']
Categories (3, object): ['1', '2', '0']

In [9]:
# .cat.categories attribute is an alternative to .unique() method
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.cat.categories.html

df.gender.cat.categories

Index(['0', '1', '2'], dtype='object')

In [10]:
# let's remove category '0' from dataframe

df = df.loc[df.gender != '0', :]

# check again unique values in 'gender'

df.gender.unique()

['1', '2']
Categories (2, object): ['1', '2']

In [11]:
# let's do a summary of percentage of users by gender wtih .value _counts() method 
# 1 >> 'Male', 2 >> 'Female'
# we will be surprise to see the category '0' is part of the summary, although result shows 0%

df.gender.value_counts(normalize=True)

1    0.739935
2    0.260065
0    0.000000
Name: gender, dtype: float64

In [12]:
# reason being, category '0' still in pandas memory although we explicitly remove gender with category '0' from dataframe

df.gender.cat.categories

Index(['0', '1', '2'], dtype='object')

### remove_categories

In [13]:
# .cat.remove_categories() helps to remove unused category from pandas memory
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.cat.remove_categories.html#pandas.Series.cat.remove_categories
# set parameter inplace = True; pandas doesn't print result of new category

df.gender.cat.remove_categories(removals = '0', inplace = True)

# category '0' no longer in pandas memory

df.gender.cat.categories

Index(['1', '2'], dtype='object')

In [14]:
# sumamrise gender percentage....again

df.gender.value_counts(normalize=True)

1    0.739935
2    0.260065
Name: gender, dtype: float64

### rename_categories

In [15]:
# category '1' & '2' are not self-explanatory
# rename the category with .cat.rename_categories() method
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.cat.rename_categories.html#pandas.Series.cat.rename_categories

df.gender.cat.rename_categories({'1': 'Male', '2': 'Female'}, inplace = True)

# new category name for gender

df.gender.cat.categories

Index(['Male', 'Female'], dtype='object')

In [16]:
# sumamrise gender percentage....again
# gender category becomes 'Male' & 'Female'

df.gender.value_counts(normalize=True)

Male      0.739935
Female    0.260065
Name: gender, dtype: float64

## Lesson 03: Feature Engineering with pandas.cut()

In bike sharing dataframe, we are given 'birth_year'. Birth year is not very useful for data analysis/visualization.

Imagine birth year range from 1970 to 2020, it is presentable in either histogram/bar chart.

In such scenario, we can perform feature engineering to group birth_year into generation. Example: anyone born between 1981 - 1996 is Generation Y. So, we can analyze bike sharing activity by generation instead of birth_year.

In [ ]:
# summary of birth_year (min & max)

df.birth_year.min(), df.birth_year.max()

(1857, 2003)

In [17]:
# let's only include birth_year from 1928 (silent generation) onward

df = df.query(" birth_year >= 1928 ").reset_index(drop = True)

In [ ]:
# total observations after exclude birth_year < 1928
# approximately 19M observations

df.shape

(18982169, 9)

### BEFORE

Apply self-defined function to 'birth_year'

In [18]:
def define_gen(year):

    gen = None
    
    if year < 1946:
        gen = 'silent'
    elif (year >= 1946) & (year < 1965):
        gen = 'baby_boomer'
    elif (year >= 1965) & (year < 1981):
        gen = 'gen_x'
    elif (year >= 1981) & (year < 1997):
        gen = 'gen_y'
    else:
        gen = 'gen_z'

    return gen

In [19]:
# this method present a huge challenge...it takes very long to perform the operation on large data set, in this case, 19M rows
# 6+ second for each observation; how much time required for 19M observations ??

%timeit df.birth_year.apply(define_gen)

1 loop, best of 5: 6.07 s per loop


### AFTER

Apply **pd.cut()** to group birth_year into 'generation'.

In [20]:
# we can achieve same result with pd.cut() method, with shorter time
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html?highlight=cut#pandas.cut

# 528 milisecond (0.528 second) for each observation; this is 11x faster than applying function to 'birth_year'

%%timeit 

pd.cut(df.birth_year, 
       bins = [1928, 1946, 1965, 1981, 1997, 2016], 
       labels = ['silent', 'baby_boomer', 'gen_x', 'gen_y', 'gen_z'], 
       include_lowest=True, ordered=False)

1 loop, best of 5: 528 ms per loop


In [21]:
# Gen Y is the largest user group, accounted for 54%

pd.cut(df.birth_year, 
       bins = [1928, 1946, 1965, 1981, 1997, 2016], 
       labels = ['silent', 'baby_boomer', 'gen_x', 'gen_y', 'gen_z'], 
       include_lowest=True, 
       ordered=False).value_counts(normalize=True)

gen_y          0.543800
gen_x          0.293460
baby_boomer    0.128881
gen_z          0.029339
silent         0.004520
Name: birth_year, dtype: float64

## Lesson 04: Numpy for better performance & Python Haversine

1. Make use of Numpy instead of pandas Series/DataFrame to optimize memory usage if index is not required.

2. Haversine == > A python library to calculate distance between 2 geometry coordinates.

### BEFORE: Creation of Coordinates with Tuple & List

In [22]:
# produce coordinates of start_station
# memory usage: +/- 5GB

%%memit

list(zip(df.station_latitude_start.values, df.station_longitude_start.values))

peak memory: 5406.20 MiB, increment: 2562.55 MiB


In [23]:
# produce coordinates of end_station
# memory usage: +/- 5GB

%%memit

list(zip(df.station_latitude_end.values, df.station_longitude_end.values))

peak memory: 5378.70 MiB, increment: 2633.12 MiB


### AFTER: Creation of Coordinates with Numpy & Haversine

In [24]:
# produce coordinates of start_station
# memory usage: +/- 3GB

%%memit

np.column_stack((df.station_latitude_start.values, df.station_longitude_start.values))

peak memory: 2746.66 MiB, increment: 0.00 MiB


In [25]:
# produce coordinates of end_station
# memory usage: +/- 3GB

%%memit

np.column_stack((df.station_latitude_end.values, df.station_longitude_end.values))

peak memory: 2746.66 MiB, increment: 0.00 MiB


In [26]:
# function to calculate distance of 2 coordinates

def measure_distance(lat_1, long_1, lat_2, long_2):

    # use numpy .column_stack() instead of list to optimize code operation
    # ref: https://numpy.org/doc/stable/reference/generated/numpy.column_stack.html
    coordinate_start = np.column_stack((lat_1, long_1))
    coodrinate_end = np.column_stack((lat_2, long_2))
    
    # ref: https://pypi.org/project/haversine/
    distance = haversine_vector(coordinate_start, coodrinate_end, Unit.KILOMETERS)

    return distance

In [27]:
# produce coordinates & calculate distance with function at one go
# measure memory usage : +/- 3GB
# conclusion == > it is more memory efficient to wrap everything in a function with Numpy

%%memit

measure_distance(df.station_latitude_start.values, 
                 df.station_longitude_start.values, 
                 df.station_latitude_end.values, 
                 df.station_longitude_end.values)

peak memory: 2766.87 MiB, increment: 20.20 MiB


In [28]:
# calculate distance

df['distance'] = measure_distance(df.station_latitude_start.values, 
                                  df.station_longitude_start.values, 
                                  df.station_latitude_end.values, 
                                  df.station_longitude_end.values)

In [29]:
df.distance.values

array([1.0663115, 0.5777477, 2.034302 , ..., 1.9097081, 1.0663526,
       5.004961 ], dtype=float32)

## Lesson 05: Numpy for Mathematical Operations

We can perform mathematical operation on Pandas Series. However, *Numpy* is more optimal (i.e. faster) than Pandas *Series* when dealing with large data set.

### BEFORE: Calculate Speed with Pandas Series

In [ ]:
# at the moment, 'duration' is in seconds & distance in kilometer
# we need to calculate 'speed' in kilometer/hour

# let's measure time taken to calculate 'speed' with pandas Series
# remember: 1 hour ==> 3,600 seconds

%timeit df.distance / (df.duration / (60 * 60))

10 loops, best of 5: 97.1 ms per loop


### AFTER: Calculate Speed with Numpy Array

In [ ]:
'''
measure time taken to calculate 'speed' with Numpy array
Numpy divide ==> https://numpy.org/doc/stable/reference/generated/numpy.divide.html#:~:text=divide,-numpy.&text=Returns%20a%20true%20division%20of,this%20returns%20a%20true%20division.

reason for better performance: Numpy array is more memory-efficient
ref: https://www.geeksforgeeks.org/difference-between-pandas-vs-numpy/

'''

# turn duration from 'seconds' to 'hour' with Numpy
# remember: 1 hour ==> 3,600 seconds
hour = np.divide(df.duration, 3600)

%timeit np.divide(df.distance.values, hour)

10 loops, best of 5: 46.8 ms per loop


## Lesson 06: Create/Replace Variable with pandas.assign()

In pandas, we tend to use *df['variable'] = pandas action* to create/replace a variable. This method is used multiple times if we want to create few variables: 

*df['var_01'] = action_01* <br>
*df['var_02'] = action_02* <br>
*df['var_03'] = action_03*

With *.assign()* method, we can create multiple variables with one command.
Check documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html). 

By the way, assign() method is inspired by R Tidyverse *mutate()*. Refer [here](https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_r.html#transforming) for more detail.

In [ ]:
# reengineer 'birth_year', 'duration' & 'speed' with .assign() and measure time taken
# turn 'birth_year' into 'generation'
# turn 'duration' from seconds to minutes
# create variable 'speed'

%%timeit

df.assign(generation = pd.cut(df.birth_year, bins = [1928, 1946, 1965, 1981, 1997, 2016], labels = ['silent', 'baby_boomer', 'gen_x', 'gen_y', 'gen_z'], include_lowest=True, ordered=False), 
          duration = np.divide(df.duration.values, 60), 
          speed = np.divide(df.distance.values, hour))

1 loop, best of 5: 857 ms per loop


In [30]:
hour = np.divide(df.duration, 3600)

# create/replace variable with one command
df = df.assign(generation = pd.cut(df.birth_year, bins = [1928, 1946, 1965, 1981, 1997, 2016], labels = ['silent', 'baby_boomer', 'gen_x', 'gen_y', 'gen_z'], include_lowest=True, ordered=False), 
                duration = np.divide(df.duration.values, 60), 
                speed = np.divide(df.distance.values, hour))

df.head()

,duration,time_start,station_latitude_start,station_longitude_start,station_latitude_end,station_longitude_end,user_type,birth_year,gender,distance,generation,speed
0,5.333333,2019-01-01 00:01:47.401,40.778969,-73.973747,40.788219,-73.970413,Subscriber,1971,Male,1.066311,gen_x,11.996004
1,5.266667,2019-01-01 00:04:43.736,40.751873,-73.977707,40.747803,-73.973442,Subscriber,1964,Male,0.577748,baby_boomer,6.581936
2,9.850000,2019-01-01 00:06:03.997,40.785248,-73.976677,40.773144,-73.958565,Subscriber,1987,Male,2.034302,gen_y,12.391687
3,45.316667,2019-01-01 00:07:03.545,40.732220,-73.981659,40.738045,-73.996429,Subscriber,1990,Male,1.403270,gen_y,1.857952
4,5.050000,2019-01-01 00:07:35.945,40.727436,-73.993790,40.738274,-73.987518,Subscriber,1979,Male,1.316004,gen_x,15.635686


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18982169 entries, 0 to 18982168
Data columns (total 12 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   duration                 float64       
 1   time_start               datetime64[ns]
 2   station_latitude_start   float32       
 3   station_longitude_start  float32       
 4   station_latitude_end     float32       
 5   station_longitude_end    float32       
 6   user_type                category      
 7   birth_year               int32         
 8   gender                   category      
 9   distance                 float32       
 10  generation               category      
 11  speed                    float64       
dtypes: category(3), datetime64[ns](1), float32(5), float64(2), int32(1)
memory usage: 923.2 MB


## Lesson 07: Method Chaining

Proper method chaining improves code readability and cut down intermediate variable assignment; this is especially useful when we need to perform multiple actions at one go.

From previous lessons, we created new variables (distance, generation, and speed) & reengineer duration (from seconds to minutes). Now we observe 'duration', 'distance' and 'speed' are numerical variables with more than 4 decimals. Variables such as latitude & longitude and birth_year are no longer required for analysis.

We can remove & format variables with *Method Chaining*

### BEFORE

In [ ]:
'''
Run code for demo only

Method 01: Long method chaining

1. to remove variables
2. round variables in float type to 2 decimals
3. change data type of 'speed' from 'float64' to 'float32'

Code is hardly readable !!!
'''

df.drop(columns=['station_latitude_start', 'station_longitude_start', 'station_latitude_end', 'station_longitude_end', 'birth_year']).round(2).astype({'speed': np.float32}).head()

,duration,time_start,user_type,gender,distance,generation,speed
0,5.33,2019-01-01 00:01:47.401,Subscriber,Male,1.07,gen_x,12.00
1,5.27,2019-01-01 00:04:43.736,Subscriber,Male,0.58,baby_boomer,6.58
2,9.85,2019-01-01 00:06:03.997,Subscriber,Male,2.03,gen_y,12.39
3,45.32,2019-01-01 00:07:03.545,Subscriber,Male,1.40,gen_y,1.86
4,5.05,2019-01-01 00:07:35.945,Subscriber,Male,1.32,gen_x,15.64


In [ ]:
'''
Run code for demo only

Method 02: Variable reassignment for each step

1. to remove variables
2. round variables in float type to 2 decimals
3. change data type of 'speed' from 'float64' to 'float32'

Code is readable but we created unnecessary variables at every step
'''

# remove variables
s1 = df.drop(columns=['station_latitude_start', 'station_longitude_start', 'station_latitude_end', 'station_longitude_end', 'birth_year'])

# round numerical variables to 2 decimals
s2 = s1.round(2)

# change data type of speed to 'float32'
s3 = s2.astype({'speed': np.float32})

s3.head()

,duration,time_start,user_type,gender,distance,generation,speed
0,5.33,2019-01-01 00:01:47.401,Subscriber,Male,1.07,gen_x,12.00
1,5.27,2019-01-01 00:04:43.736,Subscriber,Male,0.58,baby_boomer,6.58
2,9.85,2019-01-01 00:06:03.997,Subscriber,Male,2.03,gen_y,12.39
3,45.32,2019-01-01 00:07:03.545,Subscriber,Male,1.40,gen_y,1.86
4,5.05,2019-01-01 00:07:35.945,Subscriber,Male,1.32,gen_x,15.64


### AFTER

In [32]:
'''
Actual Version

Method 03: Proper Method Chaining wtih back slash '\'

1. to remove variables
2. round variables in float type to 2 decimals
3. change data type of 'speed' & 'duration' from 'float64' to 'float32'

'''

df = df.drop(columns=['station_latitude_start', 'station_longitude_start', 'station_latitude_end', 'station_longitude_end', 'birth_year'])\
        .round(2)\
        .astype({'speed': np.float32, 'duration': np.float32})

df.head()

,duration,time_start,user_type,gender,distance,generation,speed
0,5.33,2019-01-01 00:01:47.401,Subscriber,Male,1.07,gen_x,12.00
1,5.27,2019-01-01 00:04:43.736,Subscriber,Male,0.58,baby_boomer,6.58
2,9.85,2019-01-01 00:06:03.997,Subscriber,Male,2.03,gen_y,12.39
3,45.32,2019-01-01 00:07:03.545,Subscriber,Male,1.40,gen_y,1.86
4,5.05,2019-01-01 00:07:35.945,Subscriber,Male,1.32,gen_x,15.64


In [33]:
# let's see the changes

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18982169 entries, 0 to 18982168
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   duration    float32       
 1   time_start  datetime64[ns]
 2   user_type   category      
 3   gender      category      
 4   distance    float32       
 5   generation  category      
 6   speed       float32       
dtypes: category(3), datetime64[ns](1), float32(3)
memory usage: 416.4 MB


## Lesson 08: Reindex

We can change order of variables with *.reindex()*. This is a formal way of reordering variables.

In [34]:
# ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html?highlight=reindex#pandas.DataFrame.reindex

df = df.reindex(columns = ['time_start', 'user_type', 'gender', 'generation', 'duration', 'distance', 'speed'])
df.head()

,time_start,user_type,gender,generation,duration,distance,speed
0,2019-01-01 00:01:47.401,Subscriber,Male,gen_x,5.33,1.07,12.00
1,2019-01-01 00:04:43.736,Subscriber,Male,baby_boomer,5.27,0.58,6.58
2,2019-01-01 00:06:03.997,Subscriber,Male,gen_y,9.85,2.03,12.39
3,2019-01-01 00:07:03.545,Subscriber,Male,gen_y,45.32,1.40,1.86
4,2019-01-01 00:07:35.945,Subscriber,Male,gen_x,5.05,1.32,15.64


## Lesson 09: Time Series with resample()

*.resample()* method in pandas is very useful to organize data by hour, day, week, month etc.

### Group by Month

In [ ]:
# let's say we want to find out monthly bike usage in 2019
# ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html?highlight=resample#pandas.DataFrame.resample

# 'M' >> group by "month"
# parameter 'on' must be set to a variable with 'datetime64' data type

df.resample('M', on = 'time_start')['gender'].count()

time_start
2019-01-31     932320
2019-02-28     906456
2019-03-31    1255656
2019-04-30    1627078
2019-05-31    1755529
2019-06-30    1925806
2019-07-31    1984048
2019-08-31    2117821
2019-09-30    2237988
2019-10-31    1938000
2019-11-30    1393111
2019-12-31     908356
Freq: M, Name: gender, dtype: int64

### Group by Hour

In [ ]:
# 'H' >> group by "Hour"
# parameter 'on' must be set to a 'datetime64' variable

df.resample('H', on = 'time_start')['gender'].count()

time_start
2019-01-01 00:00:00     125
2019-01-01 01:00:00     255
2019-01-01 02:00:00     261
2019-01-01 03:00:00     179
2019-01-01 04:00:00     114
                       ... 
2019-12-31 19:00:00    1086
2019-12-31 20:00:00     677
2019-12-31 21:00:00     514
2019-12-31 22:00:00     488
2019-12-31 23:00:00     363
Freq: H, Name: gender, Length: 8760, dtype: int64

## Lesson 10: Datetime Manipulation with Pandas

In our dataframe, 'time_start' variable is in *datetime64* format. We can extract date/time with *.dt* followed by datetime attribute/method (second, hour, day, week, day name, month & month name).

In [ ]:
# example: day
# ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.day.html?highlight=day#pandas.Series.dt.day

# only use 100 observations for demo
df.time_start[0:100].dt.day

0     1
1     1
2     1
3     1
4     1
     ..
95    1
96    1
97    1
98    1
99    1
Name: time_start, Length: 100, dtype: int64

In [ ]:
# example: day name (i.e. Monday, Tuesday)
# ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.day_name.html?highlight=day_name

# only use 100 observations for demo
df.time_start[0:100].dt.day_name()

0     Tuesday
1     Tuesday
2     Tuesday
3     Tuesday
4     Tuesday
       ...   
95    Tuesday
96    Tuesday
97    Tuesday
98    Tuesday
99    Tuesday
Name: time_start, Length: 100, dtype: object

In [ ]:
# example: month name (i.e. January)
# ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.month_name.html?highlight=month_name#pandas.Series.dt.month_name

# only use 100 observations for demo
df.time_start[0:100].dt.month_name()

0     January
1     January
2     January
3     January
4     January
       ...   
95    January
96    January
97    January
98    January
99    January
Name: time_start, Length: 100, dtype: object

## Lesson 11: Pandas Groupby

Groupby can be used together with Count/Sum/Transform. It can be an alternative to *resample()* to group data by datetime variable. When done in the right way, we can combine groupby with datetime manipulation. 

For example: we can group our data set by month and count activity for each.

### Group by Month

In [35]:
# extract 'month name' from 'time_start' and group by month
# count activity of each month

activity_by_month = df.groupby(by = df.time_start.dt.month_name())['gender'].count()
activity_by_month

time_start
April        1627078
August       2117821
December      908356
February      906456
January       932320
July         1984048
June         1925806
March        1255656
May          1755529
November     1393111
October      1938000
September    2237988
Name: gender, dtype: int64

In [36]:
# current index is sort alphabetically

activity_by_month.index

Index(['April', 'August', 'December', 'February', 'January', 'July', 'June',
       'March', 'May', 'November', 'October', 'September'],
      dtype='object', name='time_start')

In [37]:
# we can change order of index with .reindex

activity_by_month = activity_by_month.reindex(['January', 'February', 'March', 'April', 'May', 'June',
                                                'July', 'August', 'September', 'October', 'November', 'December'])

activity_by_month

time_start
January       932320
February      906456
March        1255656
April        1627078
May          1755529
June         1925806
July         1984048
August       2117821
September    2237988
October      1938000
November     1393111
December      908356
Name: gender, dtype: int64

### Group by Hour

In [ ]:
# group data by "Hour" to get bike sharing activity on hourly basis
# note: time_start is in 24-hour period

df.groupby(by = df.time_start.dt.hour)['gender'].count()

time_start
0      168082
1       94773
2       57679
3       37360
4       41697
5      135311
6      431501
7      906516
8     1575432
9     1243327
10     857928
11     877483
12    1004565
13    1040427
14    1072548
15    1132770
16    1320268
17    1861592
18    1752979
19    1221752
20     833847
21     584420
22     439059
23     290853
Name: gender, dtype: int64

### Groupby & Transform

In [38]:
user_by_gender = df.groupby(by = ['user_type', 'gender'], as_index=False)['duration'].count()
user_by_gender

,user_type,gender,duration
0,Customer,Male,1045906
1,Customer,Female,585312
2,Subscriber,Male,12999422
3,Subscriber,Female,4351529


In [39]:
# function col.sum() is used to 'transform' total user by user_type when we group the dataframe by 'user_type'

user_by_gender.groupby(by = 'user_type').transform(lambda col: col.sum())

,duration
0,1631218
1,1631218
2,17350951
3,17350951


In [40]:
# we take each number in the 'col' (i.e duration), divide by the sum of each user_type group
# example: Male Customer 1,046,076 divided by total user 1,631,433 of 'Customer' user_type == > 64%

# result == > percentage of Male/Female user by user_type
# note: result is in float type, hence we use .round() method to round the number to ZERO decimal & use .astype() method to convert to integer.

user_by_gender.groupby(by = 'user_type')\
.transform(lambda col: col / col.sum() * 100)\
.round()\
.astype('int')

,duration
0,64
1,36
2,75
3,25


In [41]:
# Tip == > .apply() method only return summary of total user by user_type

user_by_gender.groupby(by = 'user_type').apply(lambda col: col.sum())

,duration
user_type,
Customer,1631218
Subscriber,17350951


## Lesson 12: String Method

Pandas String method, *.str* is an convenient way to work with Pandas Series (**NOT list or Numpy array**). The reason it is called *'string'* method: it only works for pandas Series with 'string' data type.

What can we do with string method? Some common applications for pandas Series are:

1. convert string from lower to upper case
2. slice substring from text
3. filter data
4. find & replace specific text

and many more....

Check [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#string-methods) for details

### Convert text from lower to upper case & replace string

In [ ]:
'''
In Lesson 03, we group 'birth_year' into 'generation' category. 
However, category names are in lower case and separated by underscore '_'

'''

pd.Series(df.generation.unique())

0          gen_x
1    baby_boomer
2          gen_y
3          gen_z
4         silent
dtype: category
Categories (5, object): ['gen_x', 'baby_boomer', 'gen_y', 'gen_z', 'silent']

In [ ]:
'''
with .str.title(), we convert first letter of each word to upper case
with .str.replace(), we replace underscore '_' with space ' '

reminder !!! .str only works for Series
'''

pd.Series(df.generation.unique())\
.str.title()\
.str.replace('_', ' ')

0          Gen X
1    Baby Boomer
2          Gen Y
3          Gen Z
4         Silent
dtype: object

In [ ]:
'''
note: .str is a more convenient way to the 'apply' function 'title' & 'replace'

title() >> https://www.w3schools.com/python/ref_string_title.asp
replace() >> https://www.w3schools.com/python/ref_string_replace.asp

'''

pd.Series(df.generation.unique()).apply(lambda word: word.title().replace('_', ' '))

0          Gen X
1    Baby Boomer
2          Gen Y
3          Gen Z
4         Silent
dtype: category
Categories (5, object): ['Gen X', 'Baby Boomer', 'Gen Y', 'Gen Z', 'Silent']

### Slice substring from a string

In [ ]:
'''
In Lesson 09, we group 'time_start' by month name. 
What if we only want first 3-letter from month name?

'''

activity_by_month.index

Index(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype='object', name='time_start')

In [ ]:
'''
slice first 3 letters from month name

'''

activity_by_month.index.str[0:3]

Index(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct',
       'Nov', 'Dec'],
      dtype='object', name='time_start')

In [ ]:
'''
note: .str is an alternative to apply function to slice substring based on index

'''

pd.Series(activity_by_month.index).apply(lambda word: word[0:3])

0     Jan
1     Feb
2     Mar
3     Apr
4     May
5     Jun
6     Jul
7     Aug
8     Sep
9     Oct
10    Nov
11    Dec
Name: time_start, dtype: object

### Filter data by rows

In [ ]:
'''
imagine we want to view bike sharing data for September, October, November & December

'''

activity_by_month

time_start
January       932320
February      906456
March        1255656
April        1627078
May          1755529
June         1925806
July         1984048
August       2117821
September    2237988
October      1938000
November     1393111
December      908356
Name: gender, dtype: int64

In [ ]:
'''
one thing in common for September, October, November & December >> all ends with 'ber'
filter pandas Series index with string ends with 'ber'

'''

activity_by_month.index.str.endswith('ber')

array([False, False, False, False, False, False, False, False,  True,
        True,  True,  True])

In [ ]:
# filter pandas Series with index name ends with 'ber'

activity_by_month[activity_by_month.index.str.endswith('ber')]

time_start
September    2237988
October      1938000
November     1393111
December      908356
Name: gender, dtype: int64

### Filter dataframe by columns required

In [ ]:
'''
In this dataframe, we only have 7 columns.
What if we want to check columns 'gender' & 'generation'?

'''

df.columns.values

array(['time_start', 'user_type', 'gender', 'generation', 'duration',
       'distance', 'speed'], dtype=object)

In [ ]:
'''
one thing in common for 'gender' & 'generation' >> contains 'gen' keyword
filter pandas columns index contains 'gen'
result >> True / False for matching values

'''

df.columns.str.contains('gen')

array([False, False,  True,  True, False, False, False])

In [ ]:
# filter dataframe columns with index name contains 'gen'
# note: useful for dataframe with many columns (I believe you don't want to manually type in many column names, right?)

df.loc[:, df.columns.str.contains('gen')].head()

,gender,generation
0,Male,gen_x
1,Male,baby_boomer
2,Male,gen_y
3,Male,gen_y
4,Male,gen_x


### *str* vs *for-loop + list comprehension* vs *apply + function*

In [ ]:
'''
wondering why use .str instead of for-loop + list comprehension or apply + function?
imagine you have list of month names, which NaN is part of the list...

'''

month_name = ['January', 'February', np.nan, 'April', 'May', 'June', 'July',
                'August', 'September', 'October', 'November', 'December']  

month_name

['January',
 'February',
 nan,
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [ ]:
# use of for-loop within list comprehension
# operation fail if there is NaN

[word[0:3] for word in month_name]

TypeError: ignored

In [ ]:
# apply function to slice the substring
# operation fail if there is NaN

pd.Series(month_name).apply(lambda word: word[0:3])

TypeError: ignored

In [ ]:
# .str will skip NaN item
# everything just works ...

pd.Series(month_name).str[0:3]

0     Jan
1     Feb
2     NaN
3     Apr
4     May
5     Jun
6     Jul
7     Aug
8     Sep
9     Oct
10    Nov
11    Dec
dtype: object

## Lesson 13: select_dtypes

*.select_dtypes()* is useful to subset columns by specific data type. This method comes in handy if we want to calculate correlation matrix.

In [ ]:
# imagine we want to find out correlation between variables: duration, distance & speed

df.dtypes

time_start    datetime64[ns]
user_type           category
gender              category
generation          category
duration             float32
distance             float32
speed                float32
dtype: object

In [ ]:
# data type for duration, distance & speed is 'float32'
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html

df.select_dtypes(include = np.float32).head()

,duration,distance,speed
0,5.33,1.07,12.00
1,5.27,0.58,6.58
2,9.85,2.03,12.39
3,45.32,1.40,1.86
4,5.05,1.32,15.64


In [ ]:
# .corr() method is used to obtain correlation matrix
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html?highlight=corr

df.select_dtypes(include = np.float32).corr().round(2)

,duration,distance,speed
duration,1.000000,0.061444,-0.057840
distance,0.061444,1.000000,0.306654
speed,-0.057840,0.306654,1.000000


In [ ]:
# subset columns with 'category' data type

df.select_dtypes(include = 'category').head()

,user_type,gender,generation
0,Subscriber,Male,gen_x
1,Subscriber,Male,baby_boomer
2,Subscriber,Male,gen_y
3,Subscriber,Male,gen_y
4,Subscriber,Male,gen_x
